In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [3]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [4]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [5]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [6]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.729 seconds.
Prefix dict has been built succesfully.


In [7]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [8]:
from random import shuffle
word_data[8]

['WRC',
 '汽车',
 '专用',
 '四季',
 '通用',
 '丝圈',
 '脚垫',
 '红',
 '黑色',
 '雪铁龙',
 'C3',
 'XR',
 '世嘉三厢',
 'C5',
 'C4L']

## 标签分离(一个标签分成三个标签)

In [9]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [10]:
len(data_y["label1"].unique())

22

In [11]:
len(data_y["label2"].unique())

191

In [12]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [13]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [14]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

In [15]:
from keras.preprocessing.text import *
from keras.preprocessing import sequence
tok = Tokenizer()
tok.fit_on_texts(word_data)

In [16]:
tok.word_index

{'新款': 1,
 '1': 2,
 '汽车': 3,
 '时尚': 4,
 '黑色': 5,
 '鞋': 6,
 '2016': 7,
 '休闲': 8,
 '款': 9,
 '专用': 10,
 '2': 11,
 '5': 12,
 '男士': 13,
 '跟': 14,
 '手机': 15,
 '适用': 16,
 '女': 17,
 '套装': 18,
 '男': 19,
 '韩版': 20,
 '夏季': 21,
 '儿童': 22,
 '3': 23,
 '包': 24,
 '于': 25,
 '透气': 26,
 '壳': 27,
 '套': 28,
 '休闲鞋': 29,
 '米': 30,
 '户外': 31,
 '男鞋': 32,
 '真皮': 33,
 '垫': 34,
 '红色': 35,
 '6': 36,
 '4': 37,
 '脚垫': 38,
 '女鞋': 39,
 '男女': 40,
 '白色': 41,
 '全': 42,
 '蓝色': 43,
 '创意': 44,
 '宝宝': 45,
 '防水': 46,
 '简约': 47,
 '四季': 48,
 '女士': 49,
 '8': 50,
 '保护套': 51,
 '凉鞋': 52,
 '牛皮': 53,
 '四件套': 54,
 '皮鞋': 55,
 '系列': 56,
 '专车': 57,
 '情侣': 58,
 '包围': 59,
 '灯': 60,
 '通用': 61,
 '纯棉': 62,
 '加厚': 63,
 '小': 64,
 '运动': 65,
 '单鞋': 66,
 '商务': 67,
 '学生': 68,
 '大': 69,
 '2015': 70,
 '送': 71,
 '潮流': 72,
 '客厅': 73,
 '婴儿': 74,
 '带': 75,
 '新': 76,
 '卡通': 77,
 '号': 78,
 '全棉': 79,
 '英伦': 80,
 '7': 81,
 '0': 82,
 '寸': 83,
 '可': 84,
 '现代': 85,
 '鞋子': 86,
 'led': 87,
 '单肩': 88,
 '玩具': 89,
 '黑': 90,
 '被': 91,
 '座垫': 92,
 '39': 93,
 '礼物': 94,

In [17]:
len(tok.word_index)

274420

In [18]:
x_train = tok.texts_to_sequences(word_data_train)

In [19]:
x_test = tok.texts_to_sequences(word_data_test)

In [20]:
x_train = sequence.pad_sequences(x_train, maxlen=20)
x_test  = sequence.pad_sequences(x_test, maxlen=20) 

In [21]:
x_test

array([[    0,     0,     0, ...,  7750, 27959, 72790],
       [    0,     0, 32342, ...,     2,    12,   123],
       [    0,     0, 18533, ...,    47,     4,  3167],
       ...,
       [    0,     0,     0, ...,  5760,  1549,   179],
       [    1,  1025,   497, ...,   611,    35,   169],
       [    0,     0,  8268, ...,   376,   143,   305]])


## GRU Model

In [82]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers
class GruModel(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, 
                 sequence_length, 
                 num_classes, 
                 embedding_size,  
                 num_filters, 
                 l2_reg_lambda=0.0, 
                 n_layer = 1, 
                 hidden_size = 32, 
                 batch_size = 256, 
                 vac_size = 27440):
        """
        sequence_length : 一个句子的长度（词的个数）
        embedding_size : 词向量的长度
        num_classes : 三个标签的类别数
        vac_size : 词的个数
        """
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        self.n_layer = n_layer
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        
        with tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([274420, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)

        l2_loss = tf.constant(0.0)
        
        self.embedded_input = tf.layers.batch_normalization(self.embedded_chars, training=is_training)
        self.embedded_input = tf.expand_dims(self.embedded_input, -1)
        
        with tf.variable_scope("cnn1"):
            filter_shape = [3, 3, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            self.conv_output = tf.nn.conv2d(
                    self.embedded_input,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
            self.conv_output = tf.nn.dropout(self.conv_output, self.keep_prob)
            
        #gru模型
        with tf.variable_scope('inception_text'):
            self.output = self.text_inception(self.conv_output)
        
        with tf.variable_scope("cnn2"):
            filter_shape = [5, 5, 16, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            self.conv_output = tf.nn.conv2d(
                    self.output,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            self.conv_output = tf.nn.dropout(self.conv_output, self.keep_prob)
        print(self.conv_output)
        self.h_drop = tf.reshape(self.conv_output,(-1, 16 * 20 * 4))
        print(self.h_drop.shape)
        num_filters_total = self.h_drop.shape[1]
        
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)
            
    def text_inception(self, inputs):
        output = []
        with tf.name_scope("conv-1"):
            filter_shape = [1, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            output.append(conv)
        with tf.name_scope("conv-2"):
            filter_shape1 = [1, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape1, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv1 = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            filter_shape2 = [3, 3, num_filters, 2 * num_filters]
            W1 = tf.Variable(tf.truncated_normal(filter_shape2, stddev=0.1), name="W")
            b1 = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv2 = tf.nn.conv2d(
                    conv1,
                    W1,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")   
            output.append(conv2)
        with tf.name_scope("conv-4"):
            filter_shape = [3, 3, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
            output.append(conv)
        print(tf.concat(output, 3).shape)
        return tf.concat(output, 3)

In [83]:

# batch 生成函数
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

## 参数

In [84]:
# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
sequence_len = 20

# batch 大小
batch_size = 256 

# 迭代次数
epochs = 100

is_training = True

# 词向量长度
embedding_dims = 24

# gru  的filters
num_filters = 4

# filter 的大小
filter_size = [1, 2, 3, 4, 5]

# 三个标签的类数
num_classes = [22, 191, 1192]

In [85]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn



FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto()
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = GruModel(sequence_length = sequence_len, 
                          num_classes = num_classes, 
                          embedding_size = embedding_dims, 
                          num_filters = num_filters)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3,
                  cnn.keep_prob: 0.9
                }

#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), batch_size, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev,
                      cnn.keep_prob: 1
                    }
                    if len(x_batch_dev) < batch_size:
                        continue
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                if len(x_batch) < batch_size:
                    continue
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [86]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)

(?, 20, 24, 16)
Tensor("cnn2/dropout/mul:0", shape=(?, 16, 20, 4), dtype=float32)
(?, 1280)
2019-03-02T16:44:26.055944: step 40, loss 4.1152, acc [0.39746094 0.32753906 0.23457031 0.17675781]
2019-03-02T16:44:28.508144: step 80, loss 2.93779, acc [0.58476562 0.49755859 0.40205078 0.32089844]
2019-03-02T16:44:30.908728: step 120, loss 2.39895, acc [0.67626953 0.58769531 0.50390625 0.41230469]
2019-03-02T16:44:33.259769: step 160, loss 2.02526, acc [0.74150391 0.64541016 0.56494141 0.47841797]
2019-03-02T16:44:35.593943: step 200, loss 1.84884, acc [0.77402344 0.68017578 0.59638672 0.51464844]

Evaluation:
2019-03-02T16:44:45.378536: step 200, loss 1.67541, acc [0.80206029 0.71077896 0.62220064 0.55438537] 

2019-03-02T16:44:48.258806: step 240, loss 1.63392, acc [0.79960937 0.70732422 0.63212891 0.54765625]
2019-03-02T16:44:50.937700: step 280, loss 1.57393, acc [0.81386719 0.72080078 0.64296875 0.56386719]
2019-03-02T16:44:53.618578: step 320, loss 1.4657, acc [0.82675781 0.74082031 0.

2019-03-02T16:48:58.773789: step 2600, loss 0.433343, acc [0.94466858 0.912613   0.87913584 0.83776992] 

2019-03-02T16:49:01.338604: step 2640, loss 0.552885, acc [0.92939453 0.88857422 0.85068359 0.79902344]
2019-03-02T16:49:03.565688: step 2680, loss 0.574283, acc [0.92578125 0.88525391 0.84589844 0.79677734]
2019-03-02T16:49:05.892923: step 2720, loss 0.527153, acc [0.92822266 0.89140625 0.85527344 0.80576172]
2019-03-02T16:49:08.174475: step 2760, loss 0.516726, acc [0.93027344 0.89892578 0.85976562 0.80751953]
2019-03-02T16:49:10.370760: step 2800, loss 0.556211, acc [0.92910156 0.88496094 0.84960938 0.79726562]

Evaluation:
2019-03-02T16:49:18.985288: step 2800, loss 0.411729, acc [0.94822253 0.91766861 0.88399123 0.84602909] 

2019-03-02T16:49:21.391880: step 2840, loss 0.545956, acc [0.92910156 0.88720703 0.85380859 0.79902344]
2019-03-02T16:49:23.589707: step 2880, loss 0.547007, acc [0.92910156 0.88730469 0.85126953 0.796875  ]
2019-03-02T16:49:25.851910: step 2920, loss 0.5

2019-03-02T16:53:23.254189: step 5200, loss 0.288251, acc [0.96220805 0.94363744 0.92482656 0.89110913] 

2019-03-02T16:53:25.605224: step 5240, loss 0.222212, acc [0.95820313 0.94208984 0.92841797 0.87792969]
2019-03-02T16:53:27.816939: step 5280, loss 0.228498, acc [0.96044922 0.93945312 0.93183594 0.88164062]
2019-03-02T16:53:30.019127: step 5320, loss 0.222194, acc [0.96132812 0.94628906 0.92763672 0.88447266]
2019-03-02T16:53:32.278406: step 5360, loss 0.234046, acc [0.96123047 0.94306641 0.92753906 0.88203125]
2019-03-02T16:53:34.477177: step 5400, loss 0.223808, acc [0.95986328 0.94648438 0.93037109 0.88408203]

Evaluation:
2019-03-02T16:53:43.392280: step 5400, loss 0.283937, acc [0.96417023 0.94325702 0.92645837 0.89237053] 

2019-03-02T16:53:45.765141: step 5440, loss 0.225829, acc [0.96201172 0.94355469 0.93164062 0.88662109]
2019-03-02T16:53:48.044144: step 5480, loss 0.236605, acc [0.95927734 0.94130859 0.92451172 0.87529297]
2019-03-02T16:53:50.350657: step 5520, loss 0.2

2019-03-02T16:57:47.274914: step 7800, loss 0.239984, acc [0.97168857 0.95759293 0.94345724 0.91736828] 

2019-03-02T16:57:49.925037: step 7840, loss 0.102667, acc [0.97568359 0.96894531 0.96835938 0.93251953]
2019-03-02T16:57:52.211105: step 7880, loss 0.0986808, acc [0.97392578 0.96855469 0.96835938 0.93017578]
2019-03-02T16:57:54.553708: step 7920, loss 0.10207, acc [0.97636719 0.96699219 0.96669922 0.92998047]
2019-03-02T16:57:56.795132: step 7960, loss 0.110479, acc [0.97617188 0.96943359 0.96240234 0.92753906]
2019-03-02T16:57:59.114964: step 8000, loss 0.113002, acc [0.9734375  0.96845703 0.96396484 0.92910156]

Evaluation:
2019-03-02T16:58:09.233865: step 8000, loss 0.257903, acc [0.97199892 0.95780316 0.94496892 0.91922033] 

2019-03-02T16:58:11.758953: step 8040, loss 0.107196, acc [0.97451172 0.96923828 0.96669922 0.93046875]
2019-03-02T16:58:13.981526: step 8080, loss 0.104184, acc [0.97392578 0.96767578 0.96699219 0.92900391]
2019-03-02T16:58:16.228018: step 8120, loss 0.1

KeyboardInterrupt: 

In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape